In [5]:
import pandas as pd
import numpy as np
# Load the datasets
data1 = pd.read_csv('data1.csv')
data2 = pd.read_csv('data2.csv')
data3 = pd.read_csv('data3.csv')

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Merge datasets on 'id' where applicable
merged_data = pd.merge(data1, data2, on='id', how='inner')
merged_data = pd.merge(merged_data, data3, left_on='id', right_on='seqno', how='inner')

# Define numerical and categorical features
numerical_features = ['age_x', 'weight', 'height', 'bmi', 'futime']
categorical_features = ['male']

# Numerical transformer
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Categorical transformer
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers into a preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the target variable
X = merged_data.drop(columns=['status'])
y = merged_data['status']

# Apply the preprocessor to the merged data
X_processed = preprocessor.fit_transform(X)

# Convert target labels to numpy array with float32 dtype
y = np.array(y, dtype=np.float32)

# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [6]:
merged_data.head()

,Unnamed: 0_x,id,age_x,male,weight,height,bmi,case.id,futime,status,...,Unnamed: 0,seqno,instit,histol,stage,study,rel,edrel,age_y,in.subcohort
0,3631,1,57,0,60.0,163.0,22.690939,10630.0,6261,0,...,1,1,2,2,1,3,0,6075,25,False
1,3631,1,57,0,60.0,163.0,22.690939,10630.0,6261,0,...,1,1,2,2,1,3,0,6075,25,False
2,3631,1,57,0,60.0,163.0,22.690939,10630.0,6261,0,...,1,1,2,2,1,3,0,6075,25,False
3,3631,1,57,0,60.0,163.0,22.690939,10630.0,6261,0,...,1,1,2,2,1,3,0,6075,25,False
4,3631,1,57,0,60.0,163.0,22.690939,10630.0,6261,0,...,1,1,2,2,1,3,0,6075,25,False


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.initializers import HeNormal, GlorotUniform
from tensorflow.keras.optimizers import legacy

# Define the model creation function
def create_model(optimizer='adam', init='he_normal'):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu', kernel_initializer=init))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu', kernel_initializer=init))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Define legacy optimizers
optimizers = {
    'Adam': legacy.Adam(),
    'RMSprop': legacy.RMSprop(),
    'SGD': legacy.SGD()
}
initializers = ['he_normal', 'glorot_uniform']

best_accuracy = 0
best_params = {}

# Manual grid search
for opt_name, optimizer in optimizers.items():
    for init in initializers:
        model = create_model(optimizer, init)
        model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
        y_pred = (model.predict(X_test) > 0.5).astype("int32")
        accuracy = accuracy_score(y_test, y_pred)
        
        # Keep track of the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'optimizer': opt_name, 'init': init}

print(f"Best accuracy: {best_accuracy}")
print(f"Best parameters: {best_params}")




576/576 [==============================] - 1s 1ms/step
Best accuracy: 0.9591892185631996
Best parameters: {'optimizer': 'Adam', 'init': 'he_normal'}


In [5]:
# Build and train the best model
best_optimizer = optimizers[best_params['optimizer']]
best_init = best_params['init']

best_model = create_model(best_optimizer, best_init)
best_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluate on test set
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

576/576 [==============================] - 2s 3ms/step
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     16341
         1.0       0.96      0.67      0.79      2061

    accuracy                           0.96     18402
   macro avg       0.96      0.83      0.88     18402
weighted avg       0.96      0.96      0.96     18402



In [8]:
! pip install tensorflow autokeras

Defaulting to user installation because normal site-packages is not writeable
INFO: pip is looking at multiple versions of autokeras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of keras-nlp to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of keras-nlp to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ---------------------------------------- 0.0/162.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/162.4 kB ? eta -:--:--
   ------- ------------------------------- 30.7/162.4 kB 445.2 kB/s eta 0:00:01
   --------- 


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Attention, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import autokeras as ak

In [2]:
# Define the paths to your training and validation datasets
train_dir = 'C:/Users/sumit/Desktop/Deep Learning/train'  
val_dir = 'C:/Users/sumit/Desktop/Deep Learning/validation'  

In [6]:
# Function to load images and labels into NumPy arrays
def load_images_from_directory(directory, target_size=(224, 224)):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )
    
    # Extract images and labels
    images, labels = [], []
    for i in range(len(generator)):
        img, lbl = generator[i]
        images.append(img)
        labels.append(lbl)
    
    return np.vstack(images), np.vstack(labels)

# Load training and validation data
X_train, y_train = load_images_from_directory(train_dir)
X_val, y_val = load_images_from_directory(val_dir)

Found 141 images belonging to 3 classes.


Found 31 images belonging to 3 classes.


In [7]:
import autokeras as ak

# Initialize the ImageClassifier
clf = ak.ImageClassifier(
    overwrite=True,
    max_trials=2  # You can increase this for more thorough searches
)

# Train the model with NumPy arrays
clf.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Trial 2 Complete [00h 02m 30s]
val_loss: 1.1400482654571533

Best val_loss So Far: 0.4665016531944275
Total elapsed time: 00h 03m 18s
Epoch 1/10
5/5 [==============================] - 5s 845ms/step - loss: 72.4137 - accuracy: 0.1631 - val_loss: 20.9517 - val_accuracy: 0.3226
Epoch 2/10
5/5 [==============================] - 4s 767ms/step - loss: 6.4027 - accuracy: 0.6596 - val_loss: 5.6383 - val_accuracy: 0.3226
Epoch 3/10
5/5 [==============================] - 4s 819ms/step - loss: 4.2713 - accuracy: 0.2482 - val_loss: 0.4789 - val_accuracy: 0.6774
Epoch 4/10
5/5 [==============================] - 4s 830ms/step - loss: 0.7799 - accuracy: 0.6596 - val_loss: 0.5472 - val_accuracy: 0.6774
Epoch 5/10
5/5 [==============================] - 4s 813ms/step - loss: 0.8954 - accuracy: 0.6596 - val_loss: 0.6606 - val_accuracy: 0.6774
Epoch 6/10
5/5 [==============================] - 4s 867ms/step - loss: 0.6271 - accuracy: 0.5674 - val_loss: 0.5075 - val_accuracy: 0.6774
Epoch 7/10
5/5 [========

INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


In [9]:
best_model = clf.export_model()

In [16]:
for layer in best_model.layers:
    print(layer.name, layer.output_shape)

input_1 [(None, 224, 224, 3)]
cast_to_float32 (None, 224, 224, 3)
normalization (None, 224, 224, 3)
conv2d (None, 222, 222, 32)
conv2d_1 (None, 220, 220, 64)
max_pooling2d (None, 110, 110, 64)
dropout (None, 110, 110, 64)
flatten (None, 774400)
dropout_1 (None, 774400)
dense (None, 3)
classification_head_1 (None, 3)


In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Multiply, Activation, BatchNormalization, Flatten

# Get the input shape from the best model
input_shape = best_model.input_shape[1:]

# Rebuild the model with added attention layers
inputs = Input(shape=input_shape)

# Use the existing layers from the best model up to the last convolutional layer
x = inputs
for layer in best_model.layers[1:]:
    if isinstance(layer, Flatten):  # Stop before the Flatten layer
        break
    x = layer(x)

# At this point, x should have shape (None, 110, 110, 64)

# Spatial Attention mechanism
# 1. Apply a 1x1 convolution to generate an attention map
attn = Conv2D(1, (1, 1), activation='sigmoid')(x)

# 2. Multiply the attention map with the input feature map
x = Multiply()([x, attn])

# Now flatten the output
x = Flatten()(x)

# Continue with the remaining layers from the best model
for layer in best_model.layers:
    if isinstance(layer, Dense):
        x = layer(x)

# Rebuild the model
model_with_attention = Model(inputs, x)

# Compile the enhanced model
model_with_attention.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the enhanced model
model_with_attention.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
5/5 [==============================] - 6s 790ms/step - loss: 15.7752 - accuracy: 0.6525 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 2/10
5/5 [==============================] - 4s 822ms/step - loss: 16.0038 - accuracy: 0.6454 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 3/10
5/5 [==============================] - 4s 840ms/step - loss: 15.7752 - accuracy: 0.6454 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 4/10
5/5 [==============================] - 4s 756ms/step - loss: 15.7752 - accuracy: 0.6596 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 5/10
5/5 [==============================] - 4s 756ms/step - loss: 15.7752 - accuracy: 0.6667 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 6/10
5/5 [==============================] - 5s 913ms/step - loss: 15.8895 - accuracy: 0.6596 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 7/10
5/5 [==============================] - 4s 830ms/step - loss: 15.7752 - accuracy: 0.6596 - val_loss: 16.1181 - val_accuracy: 0.6774
Epoch 

In [18]:
# Evaluate on validation data
loss, accuracy = model_with_attention.evaluate(X_val, y_val)
print(f"Validation Accuracy with Attention: {accuracy:.4f}")

# Save the enhanced model
model_with_attention.save('fatty_liver_ultrasound_model_with_attention.h5')

1/1 [==============================] - 0s 233ms/step - loss: 16.1181 - accuracy: 0.6774
Validation Accuracy with Attention: 0.6774


C:\Users\sumit\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
